In [ ]:
import sounddevice as sd
import numpy as np
import csv
import time

# Define the parameters
duration = 3600  # Recording duration in seconds (1 hour)
sample_rate = 44100  # Sampling rate in Hz
channels = 1  # Mono audio

# Create an empty list to store the sound level data
sound_levels = []

# Function to calculate the decibel level
def calculate_decibel_level(audio_data):
    rms = np.sqrt(np.mean(audio_data**2))
    decibel_level = 20 * np.log10(rms)
    return decibel_level

# Callback function for recording audio
def audio_callback(indata, frames, time, status):
    if status:
        print("Error:", status)
    
    decibel_level = calculate_decibel_level(indata)
    sound_levels.append(decibel_level)

# Start recording
with sd.InputStream(callback=audio_callback, channels=channels,
                    samplerate=sample_rate):
    print(f"Recording for {duration} seconds...")
    sd.sleep(duration * 1000)  # Sleep for the recording duration

# Save the recorded data to a CSV file
timestamp = time.strftime("%Y%m%d%H%M%S")
filename = f"sound_levels_{timestamp}.csv"

with open(filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Time', 'Decibel Level (dB)'])
    for i, level in enumerate(sound_levels):
        csv_writer.writerow([i / sample_rate, level])

print(f"Data saved to {filename}")
